In [ ]:
import torch


In [4]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3050 Laptop GPU


In [2]:
#import libraries
import torch,torchvision,pdb,os
from torch import nn
from  torch.utils.data import DataLoader,Dataset
from torchvision import transforms
from torchvision.utils import make_grid
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import wandb

c:\Users\tejag\OneDrive\Desktop\face\face-image-generator-\naathing\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def show_from_tensor(tensor,num=25,wandb_log=1,name=" "):
    img=tensor.detach().cpu()
    images=make_grid(img[:num],nrow=5).permute(1,2,0)
    if (wandb_log==1 and wandb_active==1):
        wandb.log({name:wandb.Image(images).numpuy().clip(0,1)})
    
    plt.figure(figsize=(10,10))
    plt.imshow(images.clip(0,1))
    plt.show()

In [5]:
#defining hyper parameters
batch_size=128
epoch=1000
wandb_active=1
critic_cycle=5
lr=0.00001
z_dim=200
mean_gen_loss=[]
mean_critic_loss=[]
curr_step=0
device='cuda'
show_step=35
save_step=35

In [6]:
wandb.login(key="7b043259eca153894ca3a1f456f024961f4a979d")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\tejag\_netrc


wandb: Currently logged in as: gunateja951 (gunateja951-omni) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [8]:
experiment_name="face-image-generator"
wandb.init(
    project="test",
    group=experiment_name,
    config={
        "batch_size": batch_size,
        "epoch": epoch,
        "critic_cycle": critic_cycle,
        "lr": lr,
        "z_dim": z_dim,
        "mean_gen_loss": mean_gen_loss,
        "mean_critic_loss": mean_critic_loss,
        "curr_step": curr_step,
        "device": device,
        "show_step": show_step,
        "save_step": save_step
    }
)
config= wandb.config

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


In [9]:
print("WandB initialized with experiment name:", experiment_name)

WandB initialized with experiment name: face-image-generator
